<a href="https://colab.research.google.com/github/Uemerson/image-registration-using-opencv/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install imutils

In [ ]:
import cv2 
import numpy as np 
import os
import imutils
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# mount drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Reference image: img2_color
# Image to be aligned: img1_color

def homographyOpenCV(img2_color, img1_color):   
  # Convert to grayscale. 
  img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY) 
  img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY) 
  height, width = img2.shape 
    
  # Create ORB detector with 5000 features. 
  orb_detector = cv2.ORB_create(5000) 
    
  # Find keypoints and descriptors. 
  # The first arg is the image, second arg is the mask 
  #  (which is not reqiured in this case). 
  kp1, d1 = orb_detector.detectAndCompute(img1, None) 
  kp2, d2 = orb_detector.detectAndCompute(img2, None) 
    
  # Match features between the two images. 
  # We create a Brute Force matcher with  
  # Hamming distance as measurement mode. 
  matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True) 
    
  # Match the two sets of descriptors. 
  matches = matcher.match(d1, d2) 
    
  # Sort matches on the basis of their Hamming distance. 
  matches.sort(key = lambda x: x.distance) 
    
  # Take the top 90 % matches forward. 
  matches = matches[:int(len(matches)*90)] 
  no_of_matches = len(matches) 
    
  # Define empty matrices of shape no_of_matches * 2. 
  p1 = np.zeros((no_of_matches, 2)) 
  p2 = np.zeros((no_of_matches, 2)) 
    
  for i in range(len(matches)): 
    p1[i, :] = kp1[matches[i].queryIdx].pt 
    p2[i, :] = kp2[matches[i].trainIdx].pt 
    
  # Find the homography matrix. 
  homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC) 
    
  # Use this matrix to transform the 
  # colored image wrt the reference image. 
  transformed_img = cv2.warpPerspective(img1_color, 
                      homography, (width, height)) 
  
  return transformed_img

In [ ]:
def files_path05(arg):
  for p, _, files in os.walk(os.path.abspath(arg)):
    for file in files:
      print(os.path.join(p, file))

      referenceImg = cv2.imread(os.path.join(p, file))

      rotated30 = imutils.rotate_bound(referenceImg, 30)
      rotated60 = imutils.rotate_bound(referenceImg, 60)
      rotated90 = imutils.rotate_bound(referenceImg, 90)
      translate = imutils.translate(referenceImg, 25, -75)
      resize = imutils.resize(referenceImg, width=320, height=320)

      homography30 = homographyOpenCV(referenceImg, rotated30)
      homography60 = homographyOpenCV(referenceImg, rotated60)
      homography90 = homographyOpenCV(referenceImg, rotated90)
      homographyTranslate = homographyOpenCV(referenceImg, translate)
      homographyResize = homographyOpenCV(referenceImg, resize)

      fig = plt.figure(figsize=(20, 15))
      
      plt.subplot(6, 2, 1), plt.title('Imagem de referência', size=20), plt.axis('off'), plt.imshow(referenceImg)
      plt.subplot(6, 2, 2), plt.title('Rotação de 30 graus', size=20), plt.axis('off'), plt.imshow(rotated30)
      plt.subplot(6, 2, 3), plt.title('Rotação de 60 graus', size=20), plt.axis('off'), plt.imshow(rotated60)
      plt.subplot(6, 2, 4), plt.title('Rotação de 90 graus', size=20), plt.axis('off'), plt.imshow(rotated90)
      plt.subplot(6, 2, 5), plt.title('Translação x=25, y=-75', size=20), plt.axis('off'), plt.imshow(translate)
      plt.subplot(6, 2, 6), plt.title('Redimensionamento 320x320', size=20), plt.axis('off'), plt.imshow(resize)              
      plt.subplot(6, 2, 7), plt.title('Homography de 30 graus', size=20), plt.axis('off'), plt.imshow(homography30)
      plt.subplot(6, 2, 8), plt.title('Homography de 60 graus', size=20), plt.axis('off'), plt.imshow(homography60)
      plt.subplot(6, 2, 9), plt.title('Homography de 90 graus', size=20), plt.axis('off'), plt.imshow(homography90)
      plt.subplot(6, 2, 10), plt.title('Homography translação x=25, y=-75', size=20), plt.axis('off'), plt.imshow(homographyTranslate)
      plt.subplot(6, 2, 11), plt.title('Homography redimensionamento 320x320', size=20), plt.axis('off'), plt.imshow(homographyResize)
      
      plt.tight_layout()
      # plt.show()

      plt.savefig(
          os.path.join(p, file).replace('/dataset/', '/output/'), 
          bbox_inches = 'tight')
      
      fig.clf()
      plt.close(fig)
  
  print('finish')

In [ ]:
path_gdrive = '/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/' 
files_path05(path_gdrive)

/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/E1154S7I004.png
/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/E1154S7I000.png
/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/E1154S7I006.png
/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/E1154S7I003.png
/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/E1154S7I001.png
/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/E1154S7I007.png
/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/E1154S7I005.png
/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/E1154S7I002.png
/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/E1154S7I008.png
/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/E1154S7I015.png
/content/gdrive/My Drive/Colab Notebooks/dataset/samples-of-mr-images-1.0.0/E115